
# MCP — Demo rápido (Server + Client en una sola notebook)

**Objetivo:** mostrar, en ~10 minutos, cómo levantar un **Servidor MCP** con `FastMCP` y consumir sus **Tools**, **Resources** y **Prompts** desde un **Cliente MCP**.  
Funciona localmente y no requiere Claude/IDE externos.

> Basado en el SDK oficial de MCP para Python y su Quickstart. Ver referencias al final.



## 1) Instalación (una sola vez por entorno)

Ejecutá la celda de abajo si aún no tenés el SDK de MCP. En algunos entornos puede pedirse reiniciar el kernel tras instalar.


In [ ]:

# Si ya lo tenés instalado, podés saltarte esta celda.
# Recomendado: entorno virtual (uv, venv, conda). Aquí usamos pip por simplicidad.
# Nota: ejecutá y, si lo pide, reiniciá el kernel.
# !pip install -q "mcp[cli]" nest_asyncio



## 2) Definir y lanzar un **Servidor MCP** (FastMCP)

Vamos a exponer tres capacidades:
- **Tool** `add(a, b)`: suma dos números.
- **Resource** `greeting://{name}`: devuelve un saludo dinámico.
- **Prompt** `greet_user(name, style)`: genera un *prompt* reutilizable.

Lo ejecutamos con **Streamable HTTP** en `http://127.0.0.1:8000/mcp` para que el cliente se conecte desde esta misma notebook.


## 3)  EJECUTAR 'python server_mcp.py' para levantar el servidor en http://127.0.0.1:8000/mcp


## 4) Cliente MCP: descubrimiento e invocaciones

Nos conectamos al servidor vía **Streamable HTTP**, listamos capacidades y ejecutamos:
1. `list_tools()` ➜ herramientas disponibles  
2. `call_tool("add", {"a": 5, "b": 7})`  
3. `list_resources()` y `read_resource("greeting://Eze")`  
4. `list_prompts()` y `get_prompt("greet_user", {"name": "Ada", "style": "formal"})`


In [1]:

import asyncio
from mcp import ClientSession, types
from mcp.client.streamable_http import streamablehttp_client
from pydantic import AnyUrl

async def demo_client():
    async with streamablehttp_client("http://127.0.0.1:8000/mcp") as (read_stream, write_stream, _):
        async with ClientSession(read_stream, write_stream) as session:
            # Inicialización del handshake MCP
            await session.initialize()
            print("🔗 Conectado al servidor MCP")

            # 1) Tools
            tools = await session.list_tools()
            print("🧰 Tools disponibles:", [t.name for t in tools.tools])

            # 2) Llamar tool 'add'
            result = await session.call_tool("add", arguments={"a": 5, "b": 7})
            # Contenido (texto) y/o salida estructurada
            structured = getattr(result, "structuredContent", None)
            if structured:
                print("➕ Resultado estructurado:", structured)
            # fallback: texto
            for c in result.content:
                if isinstance(c, types.TextContent):
                    print("➕ Resultado (texto):", c.text)

            # 3) Resources
            resources = await session.list_resources()
            print("📚 Resources:", [r.uri for r in resources.resources])
            greeting = await session.read_resource(AnyUrl("greeting://Eze"))
            for c in greeting.contents:
                if isinstance(c, types.TextContent):
                    print("📖 greeting://Eze ->", c.text)

            # 4) Prompts
            prompts = await session.list_prompts()
            print("📝 Prompts:", [p.name for p in prompts.prompts])
            if any(p.name == "greet_user" for p in prompts.prompts):
                prompt = await session.get_prompt("greet_user", arguments={"name": "Ada", "style": "formal"})
                print("📝 get_prompt('greet_user') ->")
                for m in prompt.messages:
                    # Mostramos el contenido de los mensajes del prompt
                    if isinstance(m.content, types.TextContent):
                        print("   ", m.role, ":", m.content.text)

await demo_client()


🔗 Conectado al servidor MCP
🧰 Tools disponibles: ['add', 'multiply', 'power', 'factorial']
➕ Resultado estructurado: {'result': 12}
➕ Resultado (texto): 12
📚 Resources: [AnyUrl('info://server'), AnyUrl('docs://tools')]
📝 Prompts: ['greet_user', 'math_problem', 'explain_concept']
📝 get_prompt('greet_user') ->
    user : Por favor, escribe un saludo formal y profesional para Ada. El contexto es un servidor MCP educativo.



## 5) Referencias
- **Python SDK** oficial (ejemplos de `FastMCP`, clientes y transportes).  
- **Quickstarts** de **Server** y **Client**.

> Estas páginas se actualizan con frecuencia; revisalas para cambios en nombres de módulos y APIs.



> **Cómo detener el servidor:** este hilo está marcado como `daemon=True`, por lo que se cierra cuando reiniciás/parás el kernel. En un proyecto real, corré el servidor como proceso aparte (CLI) o servicio.
